In [2]:
#import time
#import numpy as np
#from PIL import Image
#import tflite_runtime.interpreter as tflite
import re
import os
import cv2

#cap = cv2.VideoCapture(0)

edgetpu='0' # make it '1' if Coral Accelerator is attached and use model with 'edgetpu' name



In [ ]:
#====custom model and label files==================
'''
model_dir = 'models/custom'

model = 'custom_detection_model.tflite'
#model = 'custom_detection_model_edgetpu.tflite'

label = 'custom_labels.txt'
'''
#=================================================

#====pretrained model and label files==================

model_dir = 'models/pretrained'

model='mobilenet_ssd_v2_coco_quant_postprocess.tflite'
#model='mobilenet_ssd_v2_coco_quant_postprocess_edgetpu.tflite'

#model='efficientdet_lite0.tflite'
#model='efficientdet_lite0_edgetpu.tflite'

label = 'coco_labels.txt'

#=================================================

model_path=os.path.join(model_dir,model)
label_path=os.path.join(model_dir,label)

In [ ]:
#--------------------object detection--------------------------------------------------
def detect_objects(interpreter, image, score_threshold=0.3, top_k=6):
    """Returns list of detected objects."""
    set_input_tensor(interpreter, image)
    #interpreter.invoke()
    invoke_interpreter(interpreter)
    
    global model_dir
    if (model_dir=='models/pretrained'):
      # for pre-trained models
      boxes = get_output_tensor(interpreter, 0)
      class_ids = get_output_tensor(interpreter, 1)
      scores = get_output_tensor(interpreter, 2)
      count = int(get_output_tensor(interpreter, 3))
    else:
      # for custom models made by Model Maker 
      scores = get_output_tensor(interpreter, 0)
      boxes = get_output_tensor(interpreter, 1)
      count = int(get_output_tensor(interpreter, 2))
      class_ids = get_output_tensor(interpreter, 3)
  
    
    def make(i):
        ymin, xmin, ymax, xmax = boxes[i]
        return Object(
            id=int(class_ids[i]),
            score=scores[i],
            bbox=BBox(xmin=np.maximum(0.0, xmin),
                      ymin=np.maximum(0.0, ymin),
                      xmax=np.minimum(1.0, xmax),
                      ymax=np.minimum(1.0, ymax)))

    return [make(i) for i in range(top_k) if scores[i] >= score_threshold]


import collections
Object = collections.namedtuple('Object', ['id', 'score', 'bbox'])

class BBox(collections.namedtuple('BBox', ['xmin', 'ymin', 'xmax', 'ymax'])):
    """Bounding box.
    Represents a rectangle which sides are either vertical or horizontal, parallel
    to the x or y axis.
    """
    __slots__ = ()
#--------------------------------------------------------------------

In [ ]:
#----------Loading Labels----------------------------------------------------
import re
def load_labels(path):
  """Loads the labels file. Supports files with or without index numbers."""
  
  with open(path, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    labels = {}
    for row_number, content in enumerate(lines):
      pair = re.split(r'[:\s]+', content.strip(), maxsplit=1)
      if len(pair) == 2 and pair[0].strip().isdigit():
        labels[int(pair[0])] = pair[1].strip()
      else:
        labels[row_number] = pair[0].strip()
  return labels


#--------------------------------------------------------------------------

In [ ]:
#------Making Interpreter---------------------------------------------------------
import platform

EDGETPU_SHARED_LIB = {
  'Linux': 'libedgetpu.so.1',
  'Darwin': 'libedgetpu.1.dylib',
  'Windows': 'edgetpu.dll'
}[platform.system()]
      
def make_interpreter(path, edgetpu):
    print (path,edgetpu)
    if(edgetpu=='0'):
        interpreter = tflite.Interpreter(model_path=path)
    else:
      path, *device = path.split('@')
      interpreter = tflite.Interpreter(model_path=path,experimental_delegates=[tflite.load_delegate(EDGETPU_SHARED_LIB,{'device': device[0]} if device else {})])
        
        
    print('Loading Model: {} '.format(path))
    
    return interpreter

#--------------------------------------------------------------------------

In [3]:
import tensorflow as tf
import cv2 as cv
import pathlib
import sys
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="model.tflite")
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
# input details
print(input_details)
# output details
print(output_details)
interpreter.allocate_tensors()
for file in pathlib.Path(sys.argv[1]).iterdir():
    # read and resize the image
    img = cv.imread(r"{}".format(file.resolve()))
    new_img = cv.resize(img, (224, 224))
    
    interpreter.set_tensor(input_details[0]['index'], [new_img])
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    if (output_data[0][0] > 200):
        print("file {} -> indosiar_ads".format(file.stem))
    elif (output_data[0][1] > 200):
        print("file {} -> indosiar".format(file.stem))
    elif (output_data[0][2] > 200):
        print("file {} -> sctv".format(file.stem))
    elif (output_data[0][3] > 200):
        print("file {} -> sctv_ads".format(file.stem))
    else:
        print("file {} -> unknown".format(file.stem))

ValueError: Could not open 'model.tflite'.